## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the `raw_sales` table from the database `retail_sales` fon of Ironhack's databases. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per store.
    - A table for the aggregate per item.

## Instructions
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [2]:
# your code here
import pandas as pd
import numpy as np
import os

In [4]:
os.getcwd()

'/Users/andresantaclara/Desktop/IronHack/Week2/Day3/lab-df-calculation-and-transformation/your-code'

In [17]:
raw_sales = pd.read_csv("/Users/andresantaclara/Desktop/IronHack/Week2/Day3/lab-df-calculation-and-transformation/data/retail_sales-raw_sales.csv",sep=';')

In [31]:
raw_sales

,date,shop_id,item_id,item_price,item_cnt_day
0,2015-01-04 00:00:00,29,1469,1199.0,1.0
1,2015-01-04 00:00:00,28,21364,479.0,1.0
2,2015-01-04 00:00:00,28,21365,999.0,2.0
3,2015-01-04 00:00:00,28,22104,249.0,2.0
4,2015-01-04 00:00:00,28,22091,179.0,1.0
...,...,...,...,...,...
4540,2015-01-04 00:00:00,15,4240,1299.0,1.0
4541,2015-01-04 00:00:00,14,21922,99.0,1.0
4542,2015-01-04 00:00:00,15,1969,3999.0,1.0
4543,2015-01-04 00:00:00,14,22091,179.0,1.0


In [38]:
raw_sales["revenue"] = raw_sales["item_price"] * raw_sales["item_cnt_day"]
raw_sales

,date,shop_id,item_id,item_price,item_cnt_day,revenue
0,2015-01-04 00:00:00,29,1469,1199.0,1.0,1199.0
1,2015-01-04 00:00:00,28,21364,479.0,1.0,479.0
2,2015-01-04 00:00:00,28,21365,999.0,2.0,1998.0
3,2015-01-04 00:00:00,28,22104,249.0,2.0,498.0
4,2015-01-04 00:00:00,28,22091,179.0,1.0,179.0
...,...,...,...,...,...,...
4540,2015-01-04 00:00:00,15,4240,1299.0,1.0,1299.0
4541,2015-01-04 00:00:00,14,21922,99.0,1.0,99.0
4542,2015-01-04 00:00:00,15,1969,3999.0,1.0,3999.0
4543,2015-01-04 00:00:00,14,22091,179.0,1.0,179.0


In [63]:
revenue_store = raw_sales.groupby(by="shop_id",axis=0).sum().sort_values(by="revenue",ascending=False)
revenue_store.head(3)

,item_id,item_price,item_cnt_day,revenue
shop_id,,,,
42,3004773,327864.0,249.0,330111.0
31,3896166,268098.0,402.0,304692.0
12,1647339,212196.4,216.0,295173.0


In [62]:
revenue_item = raw_sales.groupby(by=["item_id","shop_id"],axis=0).sum().sort_values(by="revenue",ascending=False)
revenue_item.head(3)

,,item_price,item_cnt_day,revenue
item_id,shop_id,,,
13442,44,83970.0,3.0,83970.0
13401,37,82176.0,3.0,82176.0
6675,50,80970.0,3.0,80970.0
